In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

import numpy as np
from collections import Counter
import re
import plotly as pl 
import matplotlib.pyplot as plt
# Import db related librarys
import psycopg2
import sys
# import boto3
import os
import sqlalchemy
import json
import csv
from getpass import getpass


In [2]:
# Configure connection parameters to AWS RDS
def connect_to_db():
    
    connection = None
    #enter_password = getpass('Enter database password')
    
    try:
        print('Connecting to the PostgreSQL database...')
        connection = psycopg2.connect(
            host = "database-chocolate.cafzzay3t2tr.us-east-2.rds.amazonaws.com",
            port = 5432,
            user = 'postgres',
            password = getpass('Enter database password'),
            database = 'postgres'
            )
        print('Connection successful')
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    return connection

In [3]:
# Call the connect_to_db to connect to database
connection = connect_to_db()

Connecting to the PostgreSQL database...
Enter database password········
Connection successful


In [4]:
# Create cursor to perform database operations
cursor = connection.cursor()

# If curser object = 0, then connection was successfully established
cursor

<cursor object at 0x000002029A50D588; closed: 0>

In [5]:
# Query the location_table from AWS database and store in dataframe - verify there is data in the table.
sql = """
SELECT * FROM location_table
"""

location_table_from_db = pd.read_sql(sql, con=connection)

In [6]:
# Display the location_table queried from database
location_table_from_db

,country_code,latitude,longitude,broad_bean_origin_country
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla
...,...,...,...,...
242,ZM,-13.133897,27.849332,Zambia
243,ZW,-19.015438,29.154857,Zimbabwe
244,HI,19.898682,-155.665857,Hawaii
245,ZZ,0.000000,0.000000,Unknown


In [7]:
# Query the clean_flavours_table from AWS database and store in dataframe.
sql = """
SELECT * FROM clean_flavors_table
"""

clean_flavors_from_db = pd.read_sql(sql, con=connection)

In [8]:
clean_flavors_from_db

,company,bean_origin_or_bar_name,REF,review_date,cocoa_percent,company_location,rating,bean_type,broad_bean_origin_country,ingredients,most_memorable_characteristics,continent
0,A. Morin,Agua Grande,1876,2016,63.0,France,3.75,missing,Sao Tome & Principe,"4- B,S,C,L","sweet, chocolatey, vegetal",Africa
1,A. Morin,Kpime,1676,2015,70.0,France,2.75,missing,Togo,"4- B,S,C,L","burnt wood, earthy, choco",Africa
2,A. Morin,Atsane,1676,2015,70.0,France,3.00,missing,Togo,"4- B,S,C,L","roasty, acidic, nutty",Africa
3,A. Morin,Akata,1680,2015,70.0,France,3.50,missing,Togo,"4- B,S,C,L","mild profile, chocolaty, spice",Africa
4,A. Morin,Quilla,1704,2015,70.0,France,3.50,missing,Peru,"4- B,S,C,L","grainy texture, cocoa, sweet",South America
...,...,...,...,...,...,...,...,...,...,...,...,...
1955,Zotter,Peru,647,2011,70.0,Austria,3.75,missing,Peru,"4- B,Sw,C,Sa","creamy, fatty, mild nutty",South America
1956,Zotter,Congo,749,2011,65.0,Austria,3.00,Forastero,Republic of Congo,"4- B,Sw,C,Sa","dairy, salt, caramel",Africa
1957,Zotter,Kerala State,749,2011,65.0,Austria,3.50,Forastero,India,"4- B,Sw,C,Sa","creamy, masculine, earthy",Asia
1958,Zotter,Kerala State,781,2011,62.0,Austria,3.25,missing,India,"4- B,Sw,C,Sa","oily, subdued, caramel, salt",Asia


In [9]:
# Join the 2 tables together on broad_bean_origin_country, select the columns you want to view in df

sql = """ SELECT clean_flavors_table .*, location_table.country_code,location_table.longitude, location_table.latitude
FROM clean_flavors_table
LEFT JOIN location_table
ON clean_flavors_table.broad_bean_origin_country = location_table.broad_bean_origin_country;
"""

# Store the joined tables in dataframe
df_chocolate = pd.read_sql(sql, con=connection)

# View the new dataframe combined from two sql tables
df_chocolate.head(10)

,company,bean_origin_or_bar_name,REF,review_date,cocoa_percent,company_location,rating,bean_type,broad_bean_origin_country,ingredients,most_memorable_characteristics,continent,country_code,longitude,latitude
0,A. Morin,Agua Grande,1876,2016,63.0,France,3.75,missing,Sao Tome & Principe,"4- B,S,C,L","sweet, chocolatey, vegetal",Africa,ST,6.613081,0.186360
1,A. Morin,Kpime,1676,2015,70.0,France,2.75,missing,Togo,"4- B,S,C,L","burnt wood, earthy, choco",Africa,TG,0.824782,8.619543
2,A. Morin,Atsane,1676,2015,70.0,France,3.00,missing,Togo,"4- B,S,C,L","roasty, acidic, nutty",Africa,TG,0.824782,8.619543
3,A. Morin,Akata,1680,2015,70.0,France,3.50,missing,Togo,"4- B,S,C,L","mild profile, chocolaty, spice",Africa,TG,0.824782,8.619543
4,A. Morin,Quilla,1704,2015,70.0,France,3.50,missing,Peru,"4- B,S,C,L","grainy texture, cocoa, sweet",South America,PE,-75.015152,-9.189967
5,A. Morin,Carenero,1315,2014,70.0,France,2.75,Criollo,Venezuela,Unknown,missing,South America,VE,-66.589730,6.423750
6,A. Morin,Cuba,1315,2014,70.0,France,3.50,missing,Cuba,"4- B,S,C,L","sliglty dry, papaya",Caribbean,CU,-77.781167,21.521757
7,A. Morin,Sur del Lago,1315,2014,70.0,France,3.50,Criollo,Venezuela,Unknown,missing,South America,VE,-66.589730,6.423750
8,A. Morin,Puerto Cabello,1319,2014,70.0,France,3.75,Criollo,Venezuela,Unknown,missing,South America,VE,-66.589730,6.423750
9,A. Morin,Pablino,1319,2014,70.0,France,4.00,missing,Peru,"4- B,S,C,L","delicate, hazelnut, brownie",South America,PE,-75.015152,-9.189967


In [10]:
# Close connection to RDS
cursor.close()

In [11]:
# Check that connection is closed where object returned = -1
cursor

<cursor object at 0x000002029A50D588; closed: -1>

# Machine Learning 



In [12]:
def bin_ratings(rating):
    if rating == 5.75: return 1
    if rating == 5.50: return 1
    if rating == 5.25: return 1
    if rating == 5.00: return 1
    
    if rating == 4.75: return 1
    if rating == 4.50: return 1
    if rating == 4.25: return 1
    if rating == 4.00: return 1
    
    if rating == 3.75: return 1
    if rating == 3.50: return 0
    if rating == 3.25: return 0
    if rating == 3.00: return 0

    if rating == 2.75: return 0
    if rating == 2.50: return 0
    if rating == 2.25: return 0
    if rating == 2.00: return 0
    
    if rating == 1.75: return 0
    if rating == 1.50: return 0
    if rating == 1.25: return 0
    if rating == 1.00: return 0
    
    #print( f"error: rating={rating} type={type(rating)}" )
    return "2"

In [13]:
df_chocolate['rating'] = df_chocolate['rating'].apply(bin_ratings)


In [14]:
def to_string(value):
    other = f"_{value}_"
    return other

In [15]:
def reduce_count_vals(df, colname, threshold):
    counts = df[colname].value_counts()
    replace_list = list(counts[counts < threshold].index)

    # Replace in dataframe
    for item in replace_list:
       df[colname] = df[colname].replace(item,"Other")

In [16]:
df_chocolate['review_date']= df_chocolate['review_date'].apply(to_string)
reduce_count_vals(df_chocolate, 'review_date', 100)

## Machine Learning!

In [17]:
# Drop rows where review_date= 2009 or 2010
# indexNames = df_chocolate[(df_chocolate["review_date"]=='_2010_') | (df_chocolate["review_date"]=='_2009_')].index
# df_chocolate.drop(indexNames, inplace=True)
# df_chocolate.head()

In [18]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df_chocolate= df_chocolate.drop(columns=["company_location","REF","bean_origin_or_bar_name","most_memorable_characteristics","latitude","longitude","country_code"], axis=1)
df_chocolate.head()

,company,review_date,cocoa_percent,rating,bean_type,broad_bean_origin_country,ingredients,continent
0,A. Morin,_2016_,63.0,1,missing,Sao Tome & Principe,"4- B,S,C,L",Africa
1,A. Morin,_2015_,70.0,0,missing,Togo,"4- B,S,C,L",Africa
2,A. Morin,_2015_,70.0,0,missing,Togo,"4- B,S,C,L",Africa
3,A. Morin,_2015_,70.0,0,missing,Togo,"4- B,S,C,L",Africa
4,A. Morin,_2015_,70.0,0,missing,Peru,"4- B,S,C,L",South America


In [19]:
df_chocolate.describe()

,cocoa_percent,rating
count,1960.000000,1960.000000
mean,71.760204,0.166327
std,6.657419,0.372469
min,42.000000,0.000000
25%,70.000000,0.000000
50%,70.000000,0.000000
75%,75.000000,0.000000
max,100.000000,1.000000


In [20]:
# Generate our categorical variable lists
chocolate_cat = df_chocolate.dtypes[df_chocolate.dtypes == 'object'].index.tolist()
chocolate_cat

['company',
 'review_date',
 'bean_type',
 'broad_bean_origin_country',
 'ingredients',
 'continent']

In [21]:


# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df_chocolate[chocolate_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(chocolate_cat)
encode_df.head()



,company_A. Morin,company_AMMA,company_Acalli,company_Adi,company_Aequare (Gianduja),company_Ah Cacao,company_Akesson's (Pralus),company_Alain Ducasse,company_Alexandre,company_Altus aka Cao Artisan,...,ingredients_Unknown,continent_Africa,continent_Asia,continent_Caribbean,continent_Central America,continent_North America,continent_Oceania,continent_Other,continent_South America,continent_Unknown
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [22]:
# Merge one-hot encoded features and drop the originals
df_chocolate = df_chocolate.merge(encode_df, left_index=True, right_index=True).drop(chocolate_cat, 1)
df_chocolate.head()

,cocoa_percent,rating,company_A. Morin,company_AMMA,company_Acalli,company_Adi,company_Aequare (Gianduja),company_Ah Cacao,company_Akesson's (Pralus),company_Alain Ducasse,...,ingredients_Unknown,continent_Africa,continent_Asia,continent_Caribbean,continent_Central America,continent_North America,continent_Oceania,continent_Other,continent_South America,continent_Unknown
0,63.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,70.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,70.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,70.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,70.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [23]:
# Split our preprocessed data into our features and target arrays
y = df_chocolate["rating"]
X = df_chocolate.drop(["rating"],1) 

# Split the preprocessed data into a training and testing dataset
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

X_train, X_test, y_train, y_test = train_test_split(X,
   y,test_size= 0.4,train_size=0.6 , random_state=42
                                                , stratify=y)

In [24]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [25]:
len(X_train_scaled[0])

523

In [26]:
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=42)
X_resampled, y_resampled = smote_enn.fit_resample(X_train_scaled, y_train)


# Logistic Regression

In [27]:
classifier = LogisticRegression(solver='lbfgs',
   max_iter=400,
   random_state=42)

In [28]:
classifier.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=400, random_state=42)

In [29]:
y_pred = classifier.predict(X_test_scaled)

In [30]:
print(accuracy_score(y_test, y_pred))

0.798469387755102


In [31]:
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual average chocolate", "Actual high rated chocolate"], columns=["Predicted average rated chocolate", "Predicted high rated chocolate"])
cm_df

,Predicted average rated chocolate,Predicted high rated chocolate
Actual average chocolate,580,74
Actual high rated chocolate,84,46


In [32]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.89      0.88       654
           1       0.38      0.35      0.37       130

    accuracy                           0.80       784
   macro avg       0.63      0.62      0.62       784
weighted avg       0.79      0.80      0.80       784



# Naive Random Oversampling


In [33]:


# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
# Instantiate the model
ros = RandomOverSampler(random_state=42)
# Resample the targets
X_resampled_Oversampling, y_resampled_Oversampling = ros.fit_resample(X_train_scaled, y_train)
Counter(y_resampled_Oversampling)



Counter({1: 980, 0: 980})

In [34]:
# from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=42)
model.fit(X_resampled_Oversampling, y_resampled_Oversampling)
# Calculate predictions
y_pred = model.predict(X_test_scaled)

C:\Users\Tata\anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [35]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.6861326746647848

In [36]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[495, 159],
       [ 50,  80]], dtype=int64)

In [37]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.91      0.76      0.62      0.83      0.68      0.47       654
          1       0.33      0.62      0.76      0.43      0.68      0.46       130

avg / total       0.81      0.73      0.64      0.76      0.68      0.47       784



# Random Forest

In [38]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=400, random_state=42, criterion='entropy', min_samples_split=2,
                                  min_samples_leaf=1,max_features=30)

# Fitting the model
rf_model = rf_model.fit(X_resampled, y_resampled)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")



 Random forest predictive accuracy: 0.714


In [39]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
accuracy_score(y_test, y_pred)

0.7142857142857143

In [40]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual average chocolate", "Actual high rated chocolate"], columns=["Predicted average rated chocolate", "Predicted high rated chocolate"])
cm_df

,Predicted average rated chocolate,Predicted high rated chocolate
Actual average chocolate,508,146
Actual high rated chocolate,78,52


In [41]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.87      0.78      0.40      0.82      0.56      0.32       654
          1       0.26      0.40      0.78      0.32      0.56      0.30       130

avg / total       0.77      0.71      0.46      0.74      0.56      0.32       784



In [42]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_



In [43]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.0879443877986906, 'cocoa_percent'),
 (0.0472002848499476, 'company_Soma'),
 (0.03842476844968845, 'bean_type_missing'),
 (0.03405021705062652, 'company_Fresco'),
 (0.03241035308691204, 'continent_South America'),
 (0.028010378877528868, 'review_date__2015_'),
 (0.026651902546591888, 'ingredients_3- B,S,C'),
 (0.024169089153442413, 'bean_type_Criollo'),
 (0.021456576071987782, 'broad_bean_origin_country_Venezuela'),
 (0.02142112785858126, 'company_Bonnat'),
 (0.018908058465228268, 'bean_type_Trinitario'),
 (0.018447863676850557, 'review_date__2009_'),
 (0.018094205391907286, 'review_date_Other'),
 (0.01796750161596956, 'ingredients_Unknown'),
 (0.017848495922573645, 'ingredients_2- B,S'),
 (0.017244084708588747, 'review_date__2016_'),
 (0.015916817262558593, 'review_date__2013_'),
 (0.015237136006880814, 'review_date__2012_'),
 (0.015142480934292218, 'broad_bean_origin_country_Dominican Republic'),
 (0.014216658094482025, 'broad_bean_origin_country_Ecuador'),
 (0.0141961426247742, '

# Balanced RandomForest Classifier

In [44]:
from imblearn.ensemble import BalancedRandomForestClassifier
brf_model= BalancedRandomForestClassifier(n_estimators=500, random_state=42, criterion='entropy', min_samples_leaf=1, max_features=30)
brf_model.fit(X_train_scaled, y_train)
y_pred_brf = brf_model.predict(X_test_scaled)

In [45]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test,y_pred_brf)

0.6666313808515644

In [46]:
# Display the confusion matrix
matrix= confusion_matrix(y_test, y_pred_brf)
print(matrix)

[[389 265]
 [ 34  96]]


In [47]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_brf))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.92      0.59      0.74      0.72      0.66      0.43       654
          1       0.27      0.74      0.59      0.39      0.66      0.45       130

avg / total       0.81      0.62      0.71      0.67      0.66      0.44       784



In [48]:
importances = brf_model.feature_importances_
importances
from operator import itemgetter
ordered_importances= sorted(zip(X.columns,importances), key=itemgetter(1), reverse=True)
ordered_importances

[('cocoa_percent', 0.1019958287721469),
 ('bean_type_missing', 0.017902828844109087),
 ('ingredients_3- B,S,C', 0.01563166253873241),
 ('review_date__2015_', 0.015407272736647942),
 ('ingredients_2- B,S', 0.015090169208232243),
 ('ingredients_Unknown', 0.014269980608653768),
 ('bean_type_Criollo', 0.014161137141689336),
 ('review_date__2012_', 0.014015277213905003),
 ('bean_type_Trinitario', 0.013986075365333061),
 ('continent_South America', 0.013651318968575684),
 ('review_date__2016_', 0.01331567712137935),
 ('review_date__2014_', 0.012907550661066577),
 ('broad_bean_origin_country_Venezuela', 0.012608220931095164),
 ('review_date_Other', 0.012525597736211976),
 ('review_date__2013_', 0.012451818239255542),
 ('review_date__2009_', 0.012087107373774933),
 ('company_Soma', 0.011570990622897586),
 ('ingredients_4- B,S,C,L', 0.011287947135138037),
 ('review_date__2011_', 0.011154454927359078),
 ('review_date__2010_', 0.010999705301778233),
 ('broad_bean_origin_country_Peru', 0.010956423

# Easy Ensemble AdaBoost Classifier


In [49]:


# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=100, random_state=42)
eec.fit(X_train_scaled, y_train)



EasyEnsembleClassifier(n_estimators=100, random_state=42)

In [50]:
y_pred_eac = brf_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred_eac)

0.6666313808515644

In [51]:


# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_eac)



array([[389, 265],
       [ 34,  96]], dtype=int64)

In [52]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_eac))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.92      0.59      0.74      0.72      0.66      0.43       654
          1       0.27      0.74      0.59      0.39      0.66      0.45       130

avg / total       0.81      0.62      0.71      0.67      0.66      0.44       784



# Support Vector Machine

In [53]:
# Create the SVM model
svm = SVC(kernel='rbf')

# Train the model
svm.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = svm.predict(X_test_scaled)
print(f" SVM model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 SVM model accuracy: 0.833


In [54]:
# Calculated the balanced accuracy score
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8329081632653061

In [55]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual average chocolate", "Actual high rated chocolate"], columns=["Predicted average rated chocolate", "Predicted high rated chocolate"])
cm_df

,Predicted average rated chocolate,Predicted high rated chocolate
Actual average chocolate,643,11
Actual high rated chocolate,120,10


In [56]:
# Print the classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.98      0.91       654
           1       0.48      0.08      0.13       130

    accuracy                           0.83       784
   macro avg       0.66      0.53      0.52       784
weighted avg       0.78      0.83      0.78       784



# GradientBoost

In [57]:
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=20,
    learning_rate=learning_rate,
    max_features=5,
    max_depth=3,
    random_state=42)
    classifier.fit(X_train_scaled, y_train)
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(
           classifier.score(
               X_train_scaled,
               y_train)))
    print("Accuracy score (validation): {0:.3f}".format(
           classifier.score(
               X_test_scaled,
               y_test)))

Learning rate:  0.05
Accuracy score (training): 0.833
Accuracy score (validation): 0.834
Learning rate:  0.1
Accuracy score (training): 0.833
Accuracy score (validation): 0.834
Learning rate:  0.25
Accuracy score (training): 0.842
Accuracy score (validation): 0.832
Learning rate:  0.5
Accuracy score (training): 0.865
Accuracy score (validation): 0.821
Learning rate:  0.75
Accuracy score (training): 0.865
Accuracy score (validation): 0.812
Learning rate:  1
Accuracy score (training): 0.862
Accuracy score (validation): 0.809


In [58]:
classifier = GradientBoostingClassifier(n_estimators=20,
   learning_rate=0.75, max_features=5, max_depth=3, random_state=42)

classifier.fit(X_train_scaled, y_train)
predictions = classifier.predict(X_test_scaled)

In [59]:
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.8125


In [60]:
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
   cm, index=["Actual 0", "Actual 1"],
   columns=["Predicted 0", "Predicted 1"]
)
display(cm_df)
print("Classification Report")
print(classification_report(y_test, predictions))

,Predicted 0,Predicted 1
Actual 0,618,36
Actual 1,111,19


Classification Report
              precision    recall  f1-score   support

           0       0.85      0.94      0.89       654
           1       0.35      0.15      0.21       130

    accuracy                           0.81       784
   macro avg       0.60      0.55      0.55       784
weighted avg       0.76      0.81      0.78       784



# Deep Learning Model

In [61]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  256
hidden_nodes_layer2 = 128
hidden_nodes_layer3 = 64
hidden_nodes_layer4 = 32
hidden_nodes_layer5 = 16
hidden_nodes_layer6 = 8
hidden_nodes_layer7 = 4


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer1, input_dim = number_input_features, activation = 'relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer2, activation = 'relu'))

# other hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer3, activation = 'relu'))
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer4, activation = 'relu'))
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer5, activation = 'relu'))
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer6, activation = 'relu'))
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer7, activation = 'relu'))


# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               134144    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 3

In [62]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [63]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics= ['accuracy'])

In [64]:
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [65]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs = 100, callbacks=[cp_callback])

Epoch 1/100
37/37 [==============================] - 1s 3ms/step - loss: 0.4430 - accuracy: 0.8316
Epoch 2/100
37/37 [==============================] - 0s 3ms/step - loss: 0.3371 - accuracy: 0.8350
Epoch 3/100
37/37 [==============================] - 0s 3ms/step - loss: 0.2589 - accuracy: 0.8597
Epoch 4/100
37/37 [==============================] - 0s 3ms/step - loss: 0.2222 - accuracy: 0.9031
Epoch 5/100
37/37 [==============================] - 0s 3ms/step - loss: 0.1887 - accuracy: 0.9167

Epoch 00005: saving model to checkpoints\weights.05.hdf5
Epoch 6/100
37/37 [==============================] - 0s 4ms/step - loss: 0.1567 - accuracy: 0.9396
Epoch 7/100
37/37 [==============================] - 0s 3ms/step - loss: 0.1428 - accuracy: 0.9388
Epoch 8/100
37/37 [==============================] - 0s 3ms/step - loss: 0.1237 - accuracy: 0.9473
Epoch 9/100
37/37 [==============================] - 0s 3ms/step - loss: 0.0960 - accuracy: 0.9600
Epoch 10/100
37/37 [==============================]

37/37 [==============================] - 0s 3ms/step - loss: 0.0130 - accuracy: 0.9923
Epoch 75/100
37/37 [==============================] - 0s 3ms/step - loss: 0.0128 - accuracy: 0.9915

Epoch 00075: saving model to checkpoints\weights.75.hdf5
Epoch 76/100
37/37 [==============================] - 0s 3ms/step - loss: 0.0129 - accuracy: 0.9915
Epoch 77/100
37/37 [==============================] - 0s 3ms/step - loss: 0.0133 - accuracy: 0.9898
Epoch 78/100
37/37 [==============================] - 0s 3ms/step - loss: 0.0130 - accuracy: 0.9915
Epoch 79/100
37/37 [==============================] - 0s 3ms/step - loss: 0.0127 - accuracy: 0.9915
Epoch 80/100
37/37 [==============================] - 0s 3ms/step - loss: 0.0128 - accuracy: 0.9906

Epoch 00080: saving model to checkpoints\weights.80.hdf5
Epoch 81/100
37/37 [==============================] - 0s 3ms/step - loss: 0.0129 - accuracy: 0.9932
Epoch 82/100
37/37 [==============================] - 0s 3ms/step - loss: 0.0122 - accuracy: 0.99

In [66]:


# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")



25/25 - 0s - loss: 2.8768 - accuracy: 0.7526
Loss: 2.876793146133423, Accuracy: 0.7525510191917419


In [67]:
nn.save("Chocolate_Ratings_ML.h5")